Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 1a SEPARATE Majority Ensembling

In [8]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
import os

In [9]:
# Loading up all the predictions data

n_rnds = 3
original_df = pd.read_csv('../Datasets/Subtask_1a/training/dev.tsv', sep='\t')
models = ['bert-large-uncased', 'roberta-large']
n_models = len(models)

epoch_string = '../2nd_try_15_epochs_large_model/eval_testing/saved_eval_test_result_1a'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())



In [10]:
# Retrieving all the predictions from the 
list_of_df = pd.DataFrame(index=range(n_rnds), columns=models)

for model in models: 
    specific_model_row = []
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_eval_test_result.tsv'
            
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        list_of_df.at[rnd, model] = particular_model_df

list_of_df

,bert-large-uncased,roberta-large
0,tweet_id \ 0 SMM4H20...,tweet_id \ 0 SMM4H20...
1,tweet_id \ 0 SMM4H20...,tweet_id \ 0 SMM4H20...
2,tweet_id \ 0 SMM4H20...,tweet_id \ 0 SMM4H20...


In [11]:
# TAKING THE MAJORITY OF DATA

majority_original_tweet_id_list = []
majority_original_sentence_list = []
majority_original_label_list = []
majority_original_numbered_label_list = []

majority_predicted_number_results = pd.DataFrame(index=range(n_rows), columns=models)
majority_predicted_results = pd.DataFrame(index=range(n_rows), columns=models)


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['tweet_id']
    original_sentence = row['tweet']
    original_label = row['class']
    
    # transferring the labels over to final list
    majority_original_tweet_id_list.append(original_tweet_id)
    majority_original_sentence_list.append(original_sentence)
    majority_original_label_list.append(original_label)
    
    # go through every models' row of data 
    
    for model in models:
        specific_row_results = []
        for rnd_num in range(n_rnds):
            # print(particular_df)
            particular_df = list_of_df.at[rnd_num, model]
            row = particular_df.loc[(particular_df['tweet_id'] == original_tweet_id)]
            
            prediction = labels_to_ids[row['class'].values[0]]
            specific_row_results.append(prediction)
        
        specific_row_results = Counter(specific_row_results)
        specific_row_results = specific_row_results.most_common(1)[0][0] 
        specific_row_label = ids_to_labels[specific_row_results]
        
        majority_predicted_results.at[index, model] = specific_row_label
        majority_predicted_number_results.at[index, model] = specific_row_results
    

In [12]:
majority_predicted_number_results

,bert-large-uncased,roberta-large
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
904,0,0
905,1,0
906,0,0
907,0,0


In [13]:
# Calculating sklearn metrics

majority_original_numbered_label_list = [labels_to_ids[label] for label in majority_original_label_list]
#majority_predicted_number_results = [labels_to_ids[label] for label in majority_predicted_results]

sep_majority_metrics = pd.DataFrame(index=['f1', 'accuracy', 'precision', 'recall'], columns=models)

for model in models: 
        sep_maj_predicted_number_results = majority_predicted_number_results[model].tolist() 
        accuracy_result = accuracy_score(majority_original_numbered_label_list, sep_maj_predicted_number_results)
        f1_result = f1_score(majority_original_numbered_label_list, sep_maj_predicted_number_results, pos_label=1, average='binary')
        precision_result = precision_score(majority_original_numbered_label_list, sep_maj_predicted_number_results, pos_label=1, average='binary')
        recall_result = recall_score(majority_original_numbered_label_list, sep_maj_predicted_number_results, pos_label=1, average='binary')
        
        sep_majority_metrics.at['f1', model] = f1_result
        sep_majority_metrics.at['accuracy', model] = accuracy_result
        sep_majority_metrics.at['precision', model] = precision_result
        sep_majority_metrics.at['recall', model] = recall_result

print(sep_majority_metrics)

# Saving results to file
os.makedirs('../2nd_try_15_epochs_large_model/eval_testing/eval_validation_statistics/majority_ensemble_separate/', exist_ok=True)

sep_majority_metrics.to_csv('../2nd_try_15_epochs_large_model/eval_testing/eval_validation_statistics/majority_ensemble_separate/sep_majority_metrics.tsv', sep='\t')





          bert-large-uncased roberta-large
f1                  0.771429      0.838235
accuracy            0.964796      0.975798
precision               0.72      0.802817
recall              0.830769      0.876923


In [14]:
# # Saving it as a dataframe

# for model in models:
#     majority_predicted_results = [ids_to_labels[id] for id in majority_predicted_number_results[model].tolist()]
#     unformatted_majority_prediction_data = pd.DataFrame(zip(majority_original_tweet_id_list, majority_original_sentence_list, majority_original_label_list, majority_predicted_results), columns=['tweet_id', 'text', 'Orig', 'label'])
#     formatted_majority_prediction_data = unformatted_majority_prediction_data.drop(columns = ['text', 'Orig'])

#     to_save_location = '../15_epochs_small_model/eval_testing/sep_majority_results/' + model + '/'
#     os.makedirs(to_save_location, exist_ok=True)
#     unformatted_to_save_location = to_save_location + 'unformatted_sep_majority.tsv'
#     formatted_to_save_location = to_save_location + 'formatted_sep_majority.tsv'
    
#     unformatted_majority_prediction_data.to_csv(unformatted_to_save_location, sep='\t', index=False)
#     formatted_majority_prediction_data.to_csv(formatted_to_save_location, sep='\t', index=False)
